In [0]:
spark.conf.set(
  "fs.azure.account.key.stcensodados.dfs.core.windows.net",
  "<your-storage-account-key>"
)

dbutils.fs.ls("abfss://raw@stcensodados.dfs.core.windows.net/ibge/")

Out[3]: [FileInfo(path='abfss://raw@stcensodados.dfs.core.windows.net/ibge/tabela10061.xlsx', name='tabela10061.xlsx', size=1308703, modificationTime=1749998828000)]

In [0]:
# Copy file to local
dbutils.fs.cp(
  "abfss://raw@stcensodados.dfs.core.windows.net/ibge/tabela10061.xlsx",
  "file:/tmp/tabela10061.xlsx"
)



Out[11]: True

In [0]:

import pandas as pd

# Read headers
df_pd = pd.read_excel("/tmp/tabela10061.xlsx", sheet_name="Tabela", skiprows=4, header=None, dtype=str)

# Renomeia as colunas manualmente com base na estrutura ideal
df_pd.columns = [
    "Local", "Sexo", "Cor_ou_raca",
    "Sem_instrucao_fund_incompleto",
    "Fundamental_medio_incompleto",
    "Medio_superior_incompleto",
    "Superior_completo"
]

# Remove possíveis linhas vazias
df_pd = df_pd.dropna(subset=["Local", "Sexo", "Cor_ou_raca"], how="all")

# Preenche Local e Sexo para baixo
df_pd[["Local", "Sexo"]] = df_pd[["Local", "Sexo"]].ffill()

# Substitui valores faltantes por vazio
df_pd = df_pd.fillna("")

# Converte para Spark DataFrame
df = spark.createDataFrame(df_pd)
display(df)


In [0]:
from pyspark.sql.functions import when, col

df_raw = df  # from previous step

columns_to_clean = df_raw.columns[3:]
for col_name in columns_to_clean:
    df_raw = df_raw.withColumn(col_name,
        when(col(col_name) == "-", None).otherwise(col(col_name).cast("double"))
    )

# fill
from pyspark.sql.window import Window
from pyspark.sql.functions import last

windowSpec = Window.orderBy("Local").rowsBetween(Window.unboundedPreceding, 0)
df_cleaned = df_raw.withColumn("Local", last("Local", True).over(windowSpec)) \
                   .withColumn("Sexo", last("Sexo", True).over(windowSpec))


In [0]:
#aqui eu crio o delta em parquet na camada prata

df_cleaned.write.format("delta").mode("overwrite").save("abfss://processed@stcensodados.dfs.core.windows.net/ibge/tabela10061/")
